<h3 style='color:green;'>Decision Trees Algorithm</h3>


We are going to explain the Decision Tree algorithm step by step, focusing on classification trees (though regression trees are similar in spirit).

Steps:

1. Start at the root node with the entire dataset.

2. Select the best attribute and a split condition to partition the data into subsets (child nodes) based on a criterion (like Gini impurity or entropy for classification, variance for regression).

3. Recursively repeat the process for each child node until a stopping condition is met (e.g., maximum depth, minimum samples per leaf, pure node).

4. Assign a class label to the leaf node (majority class in classification, average value in regression).

How splits are determined?

- For each feature, we consider possible split points (for continuous features, we sort and consider midpoints between adjacent distinct values; for categorical, we consider subsets or one-hot encoding? but note: common approach is to use a greedy search over all possible splits).

- We evaluate the quality of each split using a criterion (e.g., Gini index or information gain).

Let's define:

- Gini impurity: Gini(D) = 1 - Σ (p_i)^2, where p_i is the probability of class i in the node.

- Information gain (based on entropy): IG(D, split) = Entropy(D) - Σ [ |D_j|/|D| * Entropy(D_j) ]

where Entropy(D) = - Σ p_i * log2(p_i)

For regression, we often use variance reduction.

Step-by-step:

1. Initialize:

- Start with the entire dataset at the root.

2. For each node:

- If the stopping criteria are met (e.g., the node is pure, or the depth limit is reached, or the number of samples is below a threshold), then create a leaf node and assign it the majority class (or the mean value for regression).

- Otherwise, for each feature and each possible split point of that feature, calculate the impurity (or the information gain) of the split.

3. Choose the best split:

- The split that maximizes the information gain (or minimizes the impurity) is chosen.

4. Split the node:

- Create child nodes for each subset of the data resulting from the split.

5. Recurse:

- Apply the same process recursively on each child node.

Example for classification with Gini impurity:

Suppose we have a dataset at a node with two classes: [5 positive, 5 negative].

Gini impurity = 1 - ( (5/10)^2 + (5/10)^2 ) = 1 - (0.25+0.25) = 0.5.

Now consider a split that creates two subsets:

Subset1: [3 positive, 1 negative] -> Gini1 = 1 - ( (3/4)^2 + (1/4)^2 ) = 1 - (0.5625 + 0.0625) = 0.375.

Subset2: [2 positive, 4 negative] -> Gini2 = 1 - ( (2/6)^2 + (4/6)^2 ) = 1 - (0.1111 + 0.4444) ≈ 0.4444.

The weighted average impurity for the split: (4/10)*0.375 + (6/10)*0.4444 ≈ 0.15 + 0.26664 = 0.41664.

The reduction in impurity (Gini gain) = 0.5 - 0.41664 = 0.08336.

We would compare this gain with gains from other splits and choose the split with the highest gain.

Alternatively, we can use information gain (with entropy):

Entropy before split: - (0.5 * log2(0.5) + 0.5 * log2(0.5)) = 1.

Entropy for Subset1: - ( (3/4)*log2(3/4) + (1/4)*log2(1/4) ) ≈ - (0.75*(-0.415) + 0.25*(-2)) ≈ - ( -0.311 -0.5 ) ≈ 0.811.

Entropy for Subset2: - ( (2/6)*log2(2/6) + (4/6)*log2(4/6) ) ≈ - ( (1/3)*(-1.585) + (2/3)*(-0.585) ) ≈ - ( -0.528 -0.39 ) ≈ 0.918.

Weighted average entropy: (4/10)*0.811 + (6/10)*0.918 ≈ 0.3244 + 0.5508 = 0.8752.

Information gain = 1 - 0.8752 = 0.1248.

6. Stopping criteria:

- The node becomes pure (all samples belong to one class).

- The tree reaches the maximum depth (a hyperparameter we set).

- The number of samples in the node is below a minimum threshold for splitting.

- The improvement in the split criterion (gain) is below a threshold.

7. Pruning (optional):

- After the tree is fully grown, we can prune it to avoid overfitting (by removing branches that have little impact on the accuracy, using a validation set).

For regression:

- Instead of Gini or entropy, we use the variance of the target values in the node.

- The quality of a split is measured by the reduction in variance (or mean squared error).

- The leaf node predicts the average of the target values in that leaf.

Important:

- Decision trees are prone to overfitting, so we use hyperparameters (max_depth, min_samples_split, min_samples_leaf, etc.) to control the growth.

Let's write the step-by-step algorithm in a structured way:

Algorithm: Decision Tree Learning (Classification)

function BuildTree(node, data, depth):

1. If stopping_condition(data) is True:

node.label = majority_class(data)

return

2. best_split = None

best_gain = -infinity

3. For each feature in data.features:

For each possible split point in that feature (for continuous, sort and take midpoints; for categorical, consider each subset? but note: for categorical we often use one-vs-all or multi-way splits? but in practice, for a categorical feature with k categories, we consider 2^(k-1)-1 splits, which is expensive so we often use a greedy approach or treat as ordinal? Alternatively, for binary splits, we can consider grouping categories by the proportion of the target variable. However, common libraries like scikit-learn only support binary splits for all features and use an efficient algorithm for continuous features).

Actually, for a continuous feature:

- Sort the data by that feature.

- Consider each midpoint between adjacent values that have different target labels.

For a categorical feature, we can try to split by each category (if binary) or by grouping categories? But note: scikit-learn uses an efficient method by sorting the categories by the mean of the target (for regression) or by the class probability (for classification) and then treating it as ordinal? Then the splits are of the form: category in set S vs not in S? But the set S is contiguous in the sorted order? So we only need k-1 splits for k categories.

4. For each candidate split:

left_data, right_data = split_data(data, candidate_split)

current_gain = information_gain(data, left_data, right_data)   # or Gini gain

If current_gain > best_gain:

best_gain = current_gain

best_split = candidate_split

5. If best_gain is above a threshold (or if we found a split that improves):

node.split = best_split

left_child = new Node()

right_child = new Node()

BuildTree(left_child, data_left, depth+1)

BuildTree(right_child, data_right, depth+1)

node.children = [left_child, right_child]

6. Else:   # no split found that improves enough

node.label = majority_class(data)

How to compute information gain (using entropy) for a split?

IG(D, split) = Entropy(D) - [ (|D_left|/|D|)*Entropy(D_left) + (|D_right|/|D|)*Entropy(D_right) ]

Similarly for Gini gain:

Gini_gain = Gini(D) - [ (|D_left|/|D|)*Gini(D_left) + (|D_right|/|D|)*Gini(D_right) ]

Note: There are other criteria, e.g., CART (Classification and Regression Trees) uses Gini for classification and variance for regression.

Also note: For multi-way splits (more than two children), the formula extends naturally.

However, in practice, binary splits are common because multi-way splits can fragment the data too quickly.

Additional considerations:

- Missing values: Some algorithms (like C4.5) handle missing values by using surrogate splits or by distributing the sample to all children with weights. In scikit-learn, missing values are not supported by default.

- Efficiency: For continuous features, we can compute the gain for each split point in O(n log n) per feature (if we sort once). But note: we can compute the gain in linear time after sorting by using cumulative statistics.

- Cost complexity pruning: After building the tree, we can prune it to avoid overfitting.

Let's summarize the steps in a clear, step-by-step manner:

Step 1: Start at the root node with the entire dataset.

Step 2: Check stopping conditions for the current node:

- If all samples in the node belong to the same class, then make it a leaf node and assign that class.

- If the node depth is equal to the maximum depth (if set), then make it a leaf and assign the majority class.

- If the number of samples is less than the minimum required to split, then make it a leaf and assign the majority class.

Step 3: For each feature, find the best split point (if the feature is continuous, we consider midpoints; if categorical, we consider subsets as described).

Step 4: Among all the features and their best split points, choose the split that gives the highest information gain (or Gini gain).

Step 5: If the highest gain is less than a minimum gain threshold (if set), then stop and make it a leaf.

Step 6: Split the current node into child nodes according to the chosen split.

Step 7: Recursively apply steps 2-6 for each child node.

Step 8: Return the tree.

This is a high-level explanation. Real-world implementations (like in scikit-learn) use optimized versions of this algorithm.

<h5>Step-by-Step Explanation of the Decision Tree Algorithm</h5>

Core Idea:
Decision trees partition the feature space into regions (nodes) using hierarchical splits, aiming to maximize homogeneity (purity) in the target variable within each region. The tree grows by recursively splitting nodes until stopping criteria are met.



<h3 style='color:white;'>Step 1: Start at the Root Node</h3>

Begin with the entire dataset at the root node.

Calculate the impurity of the node (e.g., Gini index or entropy for classification; variance for regression).

Key Metrics:

Gini Index (Classification):
Lower Gini = higher purity.

Entropy (Classification):
Lower entropy = higher purity.

Variance (Regression):

<h3 style='color:white;'>Step 2: Find the Best Split</h3>

For every feature and every possible split point:

(i)Split the Data:
For numerical features: Test midpoints between sorted values.
For categorical features;

(ii)Calculate Impurity Reduction:
Information Gain (IG) = Parent impurity – Weighted child impurities:
Choose the split with highest IG (or lowest weighted child impurity).

<h3 style='color:white;'>Step 3: Split the Node
</h3>

Partition the data into left/right child nodes using the best split.

Example:
Split: Age≤30 → Left node (samples ≤ 30), Right node (samples > 30).

Purity Check: If left node has mostly class "Yes", it becomes purer.

<h3 style='color:white;'>Step 4: Recursively Repeat</h3>

Repeat Steps 1–3 for each child node.
Continue until a stopping criterion is met:

Node reaches maximum depth (hyperparameter).

Node contains ≤ min_samples_split (hyperparameter).

Impurity reduction ≤ threshold (e.g., IG < 0.001).

All samples in a node belong to one class (perfect purity).

<h3 style='color:white;'>Step 5: Assign Leaf Node Labels</h3>

When splitting stops, convert terminal nodes into leaf nodes:

Classification: Predict the majority class.

Regression: Predict the mean target value.

<h3 style='color:white;'>Key Notes</h3>

Greedy Algorithm: Selects locally optimal splits at each step (no backtracking).

Hyperparameters: Control overfitting via max_depth, min_samples_split, min_impurity_decrease.

Advantages: Interpretable, handles nonlinear relationships.

Limitations: Prone to overfitting; sensitive to small data changes (use ensembles like Random Forests).

By following these steps, decision trees iteratively partition data into purer subsets, creating a hierarchical structure for prediction.


<h3 style='color:white;'>A complete example of training a Decision Tree on a real dataset using Python, including preprocessing and evaluation.</h3>

In [ ]:
# Step 1: Import Libraries
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Step 2: Load and Explore Data
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name='species')
target_names = iris.target_names

print("Feature names:", iris.feature_names)
print("Target names:", target_names)
print("\nFirst 5 samples:\n", X.head())
print("\nClass distribution:\n", y.value_counts())

# Step 3: Preprocessing
# No missing values in Iris dataset - check with:
print("\nMissing values:\n", X.isnull().sum())

# Split into train/test sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 4: Train Decision Tree
# Using Gini impurity with max_depth=3 to prevent overfitting
model = DecisionTreeClassifier(
    criterion='gini',
    max_depth=3,
    random_state=42
)
model.fit(X_train, y_train)

# Step 5: Evaluate Model
# Predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, target_names=target_names)

print("\nTest Accuracy:", f"{accuracy:.2f}")
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

# Step 6: Visualize the Tree
plt.figure(figsize=(15, 10))
plot_tree(
    model,
    feature_names=iris.feature_names,
    class_names=target_names,
    filled=True,
    rounded=True,
    proportion=True
)
plt.title("Decision Tree for Iris Classification")
plt.show()

# Step 7: Feature Importance Analysis
feature_importances = pd.Series(
    model.feature_importances_,
    index=iris.feature_names
).sort_values(ascending=False)

print("\nFeature Importances:\n", feature_importances)
feature_importances.plot(kind='bar', title='Feature Importance')
plt.ylabel('Importance Score (0-1)')
plt.show()

<h3 style='color:white;'>Dataset Loading</h3>

Uses the built-in Iris dataset (150 samples, 4 features)

3 flower species to classify (setosa, versicolor, virginica)

<h3 style='color:white;'>Preprocessing</h3>

Checks for missing values (none in Iris)

Splits data into 80% training (120 samples) and 20% testing (30 samples)

Uses stratified sampling to maintain class balance

<h3 style='color:white;'>Model Training</h3>

Uses Gini impurity as the splitting criterion

Limits tree depth to 3 (max_depth=3) to prevent overfitting

Other hyperparameters: Default settings for min_samples_split (2) and min_samples_leaf (1)

<h3 style='color:white;'>Evaluation</h3>

Accuracy: Percentage of correct predictions

Confusion Matrix: Shows true vs. predicted classes

Classification Report: Precision, recall, and F1-score per class


<h3 style='color:white;'>Visualization</h3>

Plots the decision tree with class colors and split thresholds

Shows feature importances (petal dimensions usually dominate)

<h3 style='color:white;'>Sample Output</h3>

In [ ]:
Test Accuracy: 0.97

Confusion Matrix:
 [[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]

Classification Report:
              precision  recall  f1-score   support
      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.90      0.95        10
   virginica       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

Feature Importances:
 petal length (cm)    0.666
petal width (cm)     0.334
sepal length (cm)    0.000
sepal width (cm)     0.000

<h3 style='color:white;'>Interpretation:</h3>

High Accuracy: 97% on test data (misclassified only 1 versicolor as virginica)

Key Splits:

First split: Petal length ≤ 2.45 cm → immediately identifies setosa

Subsequent splits use petal width/length to separate versicolor/virginica

Feature Importance:

Petal dimensions account for 100% of the splitting decisions

Sepal measurements are irrelevant for this tre

<h3 style='color:white;'>Best Practises</h3>

Hyperparameter Tuning:

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}
grid_search = GridSearchCV(DecisionTreeClassifier(), params, cv=5)
grid_search.fit(X_train, y_train)
print("Best params:", grid_search.best_params_)

Handling Overfitting:

If accuracy on training is much higher than testing:

Increase min_samples_split or min_samples_leaf

Reduce max_depth

Use pruning (ccp_alpha parameter)

For Larger Datasets:

Use class_weight='balanced' for imbalanced classes

Consider binning continuous features

Use max_features to limit features per split